In [29]:
NEO4J_URI="neo4j+s://371b8b4c.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="9_2rayGknsg2qDFQWLO4I1fhKGkLPrVZEOFgVgD3-nU"

In [30]:
import os
os.environ["NEO4J_URL"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [31]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [32]:
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = groq_api_key, model_name = "Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fcbcec00910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fcbcec156d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [33]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True, exclude_types=["Genre"])
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fcbcec02ad0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fcbcec00910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fcbcec156d0>, model_name='Gemma2-9b-I

In [34]:
## Dataset Movie
movie_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row
MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
 FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [35]:
movie_query

"\nLOAD CSV WITH HEADERS FROM \n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') |\n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\n FOREACH (genre in split(row.genres, '|') |\n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n"

In [36]:
graph.query(movie_query)

[]

In [37]:
chain.graph_schema

'Node properties are the following:\nCEO {POB: STRING, name: STRING, YOB: INTEGER},Person {name: STRING},Movie {title: STRING, released: DATE, id: STRING, imdbRating: FLOAT},User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER},Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie),(:User)-[:POSTED]->(:Post),(:User)-[:FRIEND]->(:User)'

In [38]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [39]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

# Define your example prompt, now using ChatMessagePromptTemplate instead of PromptTemplate
example_prompt = ChatMessagePromptTemplate.from_template(
    "You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.\nUser input: {question}\nCypher query: {query}",
    role="user"
)

# Now create the FewShotChatMessagePromptTemplate
prompt = FewShotChatMessagePromptTemplate(
    examples=examples[:5],  # Assuming 'examples' is defined somewhere
    example_prompt=example_prompt,
    input_variables=["question", "schema"]
)

In [40]:
prompt

FewShotChatMessagePromptTemplate(examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}], input_variables=['question', 'schema'], input_types={}, partial_variables={}, example_prompt=ChatMes

In [41]:
print(prompt.format(question="Jow many artist are there?", schema="foo"))

user: You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.
User input: How many artists are there?
Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)
user: You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.
User input: Which actors played in the movie Casino?
Cypher query: MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name
user: You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.
User input: How many movies has Tom Hanks acted in?
Cypher query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)
user: You are a Neo4j expert. Given an input question, create a syntactically accurate Cypher query.
User input: List all the genres of the movie Schindler's List
Cypher query: MATCH (m:Movie {{title: 'Schindler\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name
user: You are a Neo4j expert. Given an inp

In [42]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True, cypher_prompt=prompt)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fcbcec02ad0>, cypher_generation_chain=LLMChain(verbose=False, prompt=FewShotChatMessagePromptTemplate(examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHE

In [43]:
response = chain.invoke({"query":"How many movies has Tom Hanks acted in?"})
response



> Entering new GraphCypherQAChain chain...


Generated Cypher:

Full Context:
[]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer. \n"}

In [44]:
response = chain.invoke({"query":"Which actors played in the movie Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WITH a, m
MATCH (m)-[:IN_GENRE]->(g1:Genre {name: 'Comedy'}),
     (m)-[:IN_GENRE]->(g2:Genre {name: 'Action'})
RETURN DISTINCT a.name

Full Context:
[{'a.name': 'Wesley Snipes'}, {'a.name': 'Robert Blake'}, {'a.name': 'Woody Harrelson'}, {'a.name': 'Jennifer Lopez'}, {'a.name': 'Quentin Tarantino'}, {'a.name': 'George Clooney'}, {'a.name': 'Harvey Keitel'}, {'a.name': 'Juliette Lewis'}, {'a.name': 'Bill Tung'}, {'a.name': 'Anita Mui'}]

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': 'Wesley Snipes, Robert Blake, Woody Harrelson, Jennifer Lopez, Quentin Tarantino, George Clooney, Harvey Keitel, Juliette Lewis  Bill Tung, Anita Mui \n'}

In [45]:
chain.invoke("Identify movies where directors also played a role in the film.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


{'query': 'Identify movies where directors also played a role in the film.',
 'result': "I don't know the answer. \n"}

In [49]:
chain.invoke("display the actor who acted in multiple movies")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


{'query': 'display the actor who acted in multiple movies',
 'result': "I don't know the answer. \n"}